In [1]:
#import ReadRawMHD as rR
import mapping_noRot as mappNR
import numpy as np
#import matplotlib.pyplot as plt
import SimpleITK as sitk
import time
import os
t1 = time.time()

In [2]:
# # # # # Input # # # # #
# Input imaging data
# FE version
models = ['00_L50_S50_D30', '01_L50_S50_D35', '02_L50_S50_D40', '03_L50_S50_D45', '04_L50_S50_D27',  # 0, 1, 2, 3, 4
          '10_L50_S00_D30', '11_L50_S00_D35', '12_L50_S00_D40', '13_L50_S00_D45', '14_L50_S00_D27', '15_L50_S00_D44',
          '31_L50_S50_D35', '43_L50_S00_D45',  # 11, 12
          '50_L50_S00_D30', '55_L50_S00_D30',  # 13, 14
          '63_L50_S50_D45', '74_L50_S50_D45', '75_L50_S50_D45',  '76_L50_S50_D45', '77_L50_S50_D45',  # 15, 16, 17, 18, 19
          '80_L50_S50_D45', '83_L50_S50_D45']  # 20, 21


model_code = models[20]  # FEA model name
print('Model: ' + str(model_code))
sample_code = 'S131318_L1_left'  # Experimental sample. contains number and sample name
ExpScrew = 'P'  # T or P, site from experiment. Two input files (SimScrew Ti and PEEK) will always be generated

Model: 80_L50_S50_D45


In [3]:
# Specify file locations
path_project = '/home/biomech/Documents/01_Icotec/'  # General project folder
path_ct = path_project + '01_Experiments/02_Scans/' + sample_code + '/04_Registered/'  # Folder of CT data
path_fea = path_project + '02_FEA/98_Pilots/' + sample_code + '/' + model_code + '/'  # Folder of FEA files
file_bone = [filename for filename in os.listdir(path_ct + '/') if filename.endswith('image.mhd') and str(sample_code) in filename][0]
print(file_bone)

S131318_L1_left_screw_image.mhd


In [4]:
# # # # # Input # # # # #
# Input FEA data
Input = {}
Input['Project_Folder'] = path_project
Input['FEA_loc'] = path_fea  # path to FEA files
Input['Model_Code'] = model_code  # model code (FEA model name)
Input['Screw'] = ExpScrew  # which site of scan will be processed
Input['Resolution'] = 0.0606995  # scan resolution, should be 0.0606995 for HR-pQCT II

Input['Load_mode'] = 'd'  # no effect 'd' or 'f', displacement or force controlled
## ********* EXCLUDED (line 357 in mapping.py)

Input['d_dir'] = '-'  # displ direction (negative corresponds to experiment, positive = inverse). Input: '-' or '+'
Input['d_max'] = 1  # peak displ

Input['Friction'] = 0.2  # friction between screw and bone
Input['Mapping_Diameter'] = 1  # diameter of sphere for mapping, in mm. should be larger than element size

# Check if folder exists, otherwise create it
isExist = os.path.exists(Input['FEA_loc'])
if not isExist:
    os.makedirs(Input['FEA_loc'])
    print('New directory created: ' + Input['FEA_loc'])

# Submit on cortex or ubelix?
# -> Other parameters as e-mail, runtime, memory etc. can be changed in the template file
Input['Submit'] = 'ubelix'
mappNR.write_submit(Input)

# Take screw axis or points from PMMA area
screw_axis = 1

# Write output images? segmented image and mask, for visual check
write_output = 1

In [5]:
# Write mesh input file
mappNR.write_mesh(Input)  # Original input file, path for mesh.inp


Finished extracting mesh file.


In [6]:
bone = {}
bone = mappNR.readInpBoneDummy(bone, Input)  # Read bone mesh from abaqus. Read elements, nodes
bone = mappNR.load_BVTVdata(bone, path_ct + file_bone)
isExist = os.path.exists(Input['FEA_loc'] + Input['Model_Code'] + Input['Screw'] + '_mask.mhd')
if not isExist:
    print('Creating bone mesh mask')
    bone = mappNR.boneMeshMask(bone, Input)  # Create mask from abaqus bone mesh
else:
    print('Bone mesh mask existing already')

Bone mesh mask existing already


In [39]:
# Read mask
imMask = sitk.ReadImage(Input['FEA_loc'] + Input['Model_Code'] + Input['Screw'] + '_mask.mhd')
imMask_np = np.transpose(sitk.GetArrayFromImage(imMask), [2, 1, 0])
orM = imMask.GetOrigin()
orB = bone['GreyImage'].GetOrigin()
insBefore = np.rint(abs(np.array(orB)-np.array(orM))/Input['Resolution']).astype(int)
print('insBefore: ' + str(insBefore))
dimMask = np.array(imMask.GetSize())
dimBone = np.array(bone['GreyImage'].GetSize())
insAfter = (dimBone-dimMask-insBefore).astype(int)
print('insAfter: ' + str(insAfter))
print('bone dimension: ' + str(dimBone))
print('mask dimension: ' + str(dimMask))
imMask_np_corr = imMask_np
#print(np.zeros((insBefore[0], dimMask[1], dimMask[2])))
imMask_np_corr = np.append(np.insert(imMask_np_corr, 0, np.zeros((insBefore[0], dimMask[1], dimMask[2])),0), np.zeros((insAfter[0], dimMask[1], dimMask[2], )), 0)  # HERE: use imMask_np_corr dimensions instead of dimMask!!!

print('new mask dimension: ' + str(np.array(imMask_np_corr.shape)))
# HERE: insert and append
#center = np.array([imMask_np.shape[0] / 2, imMask_np.shape[1] / 2, 0]) * imMask.GetSpacing()

insBefore: [ 94 285  34]
insAfter: [ 77  22 205]
bone dimension: [ 361  596 1063]
mask dimension: [190 289 824]
new mask dimension: [361 289 824]


In [8]:
bone['MASK'] = imMask_np
if write_output:
    img_seg=sitk.GetImageFromArray(np.transpose(bone['BVTVscaled'], [2, 1, 0]))
    img_seg.SetOrigin(bone['GreyImage'].GetOrigin())
    img_seg.SetSpacing(bone['GreyImage'].GetSpacing())
    sitk.WriteImage(img_seg, path_fea + sample_code + '_seg.mhd')
    print('Segmented image saved.')
else:
    os.remove(path_fea + model_code + ExpScrew + '_mask.mhd')
    os.remove(path_fea + model_code + ExpScrew + '_mask.raw')
del img_seg
# del bone['GreyImage']

Segmented image saved.


In [9]:
mappNR.HFE_mapping_trans(bone, Input)

... start material mapping with copying boundary layers as ghost layers
offset2COS
[ 6.  9. 45.]
cog_real
[-4.97354005 -9.17402607  0.625     ]

X, Y, Z: 

[0, -73.69954133875456]
[0, -142.90113952676057]
[2.0593521767823493, 18.53389817948088]

mask

190
289
824

imarray.shape
(361, 596, 1063)

x: -81.93681434010382
, X: 0
, xc: -81.93681434010382

y: -151.13841252810982
, Y: 0
, yc: -151.13841252810982

z: 10.296625178131615
, Z: 2.0593521767823493
, zc: 8.237273001349266


ValueError: operands could not be broadcast together with shapes (287,453,17) (116,146,17) 

In [ ]:
# Write final input file
mappNR.HFE_inp_creator(Input)
if not write_output:
    os.remove(Input['FEA_loc'] + Input['Model_Code'] + Input['Screw'] + '_elsets.inp')
    os.remove(Input['FEA_loc'] + Input['Model_Code'] + Input['Screw'] + '_materials.inp')
    os.remove(Input['FEA_loc'] + Input['Model_Code'] + '_mesh.inp')

In [ ]:
tRun = time.time()-t1
if tRun >= 3600:
    print('Execution time: ' + str(int(tRun/3600)) + ' h ' + str(int(np.mod(tRun, 3600)/60)) + ' min '+ str(round(np.mod(tRun, 60), 1)) + ' sec.')
elif tRun >= 60:
    print('Execution time: ' + str(int(tRun/60)) + ' min '+ str(round(np.mod(tRun, 60), 1)) + ' sec.')
else:
    print('Execution time: ' + str(round(tRun, 1)) + ' sec.')

In [ ]:
print(bone['Origin'])
mask_origin = [11.5/2, 17.5/2, 45]
print(mask_origin)
offset = np.array(bone['Origin']) + mask_origin
print(offset)
print(np.rint(offset/0.0607))